In [29]:
import joblib
import sagemaker
import pandas as pd
from sklearn.metrics import accuracy_score

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [31]:
session = sagemaker.Session()
bucket = session.default_bucket()
path = "monitor/heart"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [6]:
!pip uninstall scikit-learn -y
!pip install scikit-learn=='1.2.1'

Found existing installation: scikit-learn 1.2.1
Uninstalling scikit-learn-1.2.1:
  Successfully uninstalled scikit-learn-1.2.1
  Using cached scikit_learn-1.2.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached scikit_learn-1.2.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.6 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-core 0.8.2 requires scikit-learn<1.5,>=1.3.0, but you have scikit-learn 1.2.1 which is incompatible.
autogluon-features 0.8.2 requires scikit-learn<1.5,>=1.3.0, but you have scikit-learn 1.2.1 which is incompatible.
autogluon-multimodal 0.8.2 requires scikit-learn<1.5,>=1.3.0, but you have scikit-learn 1.2.1 which is incompatible.
autogluon-tabular 0.8.2 requires scikit-learn<1.5,>=1.3.0, but you have scikit-learn 1.2.1 which is incompatible.


In [15]:
model = joblib.load('model.joblib')

In [16]:
df = pd.read_csv("test.csv", header=None)

In [17]:
y_test = df.iloc[:, 0]
X_test = df.iloc[:, 1:]

In [18]:
accuracy_score(y_test, model.predict(X_test)) * 100

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


78.02197802197803

In [23]:
probabs = model.predict_proba(X_test)[:, 1]
predictions = model.predict(X_test)

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


In [24]:
results = pd.DataFrame()
results["probability"] = probabs
results["prediction"] = predictions
results["label"] = y_test

In [27]:
results.to_csv("validation_with_predictions.csv", index=None)


In [32]:
f"{path}/baseline/dataset"

'monitor/heart/baseline/dataset'

In [35]:
baseline_dataset_uri = session.upload_data("validation_with_predictions.csv",bucket,
                                           f"{path}/baseline/dataset",
                                           )

In [36]:
baseline_dataset_uri

's3://sagemaker-us-east-1-381492271173/monitor/heart/baseline/dataset/validation_with_predictions.csv'

In [37]:
%store baseline_dataset_uri

Stored 'baseline_dataset_uri' (str)


In [38]:
%store path

Stored 'path' (str)
